# Set 1
## Load data

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
from IPython.display import SVG, display
from tqdm import tqdm
from pyclustertend import hopkins,ivat
from tqdm import tqdm


import hdbscan
import umap
import pickle
import os
import itertools
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from distinctipy import distinctipy

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import MiniBatchKMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

from Commen_Functions import get_score, get_score_df, merge_Results, load_sets

In [3]:
df, df_normalized, Type_df = load_sets(1)

## Hierarchical clustering

In [4]:
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet1/Merged_results.pkl', 'rb') as f:
    results_grid = pickle.load(f)

In [5]:
results_grid['Hierarch'].sort_values('silhouette_score_euclidean', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
13,2,0.096876,0.097234,0.102570,64.930801
12,15,0.039052,0.089617,0.046065,21.237303
9,6,0.039044,0.097117,0.052476,36.334550
10,4,0.038910,0.093733,0.054215,45.541359


In [6]:
results_grid['Hierarch'].sort_values('silhouette_score_correlation', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
5,11,0.037080,0.108922,0.047856,24.547031
6,10,0.034417,0.105647,0.044849,26.079604
2,9,0.031330,0.104123,0.041963,27.852091
1,7,0.037010,0.100873,0.048823,32.871596


In [7]:
results_grid['Hierarch'].sort_values('silhouette_score_manhattan', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
13,2,0.096876,0.097234,0.102570,64.930801
10,4,0.038910,0.093733,0.054215,45.541359
9,6,0.039044,0.097117,0.052476,36.334550
3,5,0.035483,0.091507,0.049485,39.454022


In [8]:
results_grid['Hierarch'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
13,2,0.096876,0.097234,0.102570,64.930801
8,3,0.025232,0.088729,0.038095,47.205457
10,4,0.038910,0.093733,0.054215,45.541359
3,5,0.035483,0.091507,0.049485,39.454022


## K-means

In [9]:
results_grid['kMeans'].sort_values('silhouette_score_euclidean', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,n_clusters,batch_size
2,0.104322,0.099542,NaN,66.925385,3.742997,2,512
3,0.104322,0.099542,0.111301,66.925385,6.348439,2,1024
1,0.104322,0.099542,0.111301,66.925385,2.610711,2,256
0,0.102786,0.100446,0.109186,67.610557,2.326726,2,128


In [10]:
results_grid['kMeans'].sort_values('silhouette_score_correlation', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,n_clusters,batch_size
9,0.049518,0.107492,0.066756,49.714021,3.091774,4,256
11,0.050610,0.106588,0.067730,49.595017,6.594293,4,1024
10,0.050594,0.106517,0.067694,49.571423,4.164761,4,512
4,0.037555,0.101401,0.051724,52.356986,1.515158,3,128


In [11]:
results_grid['kMeans'].sort_values('silhouette_score_manhattan', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,n_clusters,batch_size
3,0.104322,0.099542,0.111301,66.925385,6.348439,2,1024
1,0.104322,0.099542,0.111301,66.925385,2.610711,2,256
0,0.102786,0.100446,0.109186,67.610557,2.326726,2,128
11,0.050610,0.106588,0.067730,49.595017,6.594293,4,1024


In [12]:
results_grid['kMeans'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,n_clusters,batch_size
0,0.102786,0.100446,0.109186,67.610557,2.326726,2,128
2,0.104322,0.099542,NaN,66.925385,3.742997,2,512
3,0.104322,0.099542,0.111301,66.925385,6.348439,2,1024
1,0.104322,0.099542,0.111301,66.925385,2.610711,2,256


## UMAP + DBSCAN

In [13]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet1/dbscan.pkl', 'rb') as f:
        results_grid['dbscan'] = pickle.load(f)

In [14]:
results_grid['dbscan'].sort_values('silhouette_score_euclidean', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
146,0.104322,0.099542,0.111301,66.925385,10.296356,25,0.1,2,100,25,0.5,eom,2
149,0.104322,0.099542,0.111301,66.925385,8.794039,25,0.1,2,100,25,1.0,leaf,2
98,0.104322,0.099542,0.111301,66.925385,9.190575,25,0.1,10,100,50,0.5,eom,2
90,0.104322,0.099542,0.111301,66.925385,9.674362,25,0.1,10,100,25,0.1,eom,2


In [15]:
results_grid['dbscan'].sort_values('silhouette_score_correlation', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
144,0.104322,0.099542,0.111301,66.925385,8.580284,25,0.1,2,100,25,0.1,eom,2
160,0.104322,0.099542,0.111301,66.925385,8.494964,25,0.1,2,100,100,1.0,eom,2
158,0.104322,0.099542,0.111301,66.925385,8.453750,25,0.1,2,100,100,0.5,eom,2
157,0.104322,0.099542,0.111301,66.925385,8.844681,25,0.1,2,100,100,0.1,leaf,2


In [16]:
results_grid['dbscan'].sort_values('silhouette_score_manhattan', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
306,0.104322,0.099542,0.111301,66.925385,8.391026,25,0.5,2,100,25,0.1,eom,2
150,0.104322,0.099542,0.111301,66.925385,8.451244,25,0.1,2,100,50,0.1,eom,2
624,0.104322,0.099542,0.111301,66.925385,9.036973,50,0.1,2,50,100,0.1,eom,2
68,0.104322,0.099542,0.111301,66.925385,8.537752,25,0.1,10,25,100,0.5,eom,2


In [17]:
results_grid['dbscan'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
306,0.104322,0.099542,0.111301,66.925385,8.391026,25,0.5,2,100,25,0.1,eom,2
144,0.104322,0.099542,0.111301,66.925385,8.580284,25,0.1,2,100,25,0.1,eom,2
624,0.104322,0.099542,0.111301,66.925385,9.036973,50,0.1,2,50,100,0.1,eom,2
160,0.104322,0.099542,0.111301,66.925385,8.494964,25,0.1,2,100,100,1.0,eom,2


## HDDC

In [18]:
results_grid['hddc'].sort_values('silhouette_score_euclidean', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
6,2,0.105046,0.100257,0.111406,67.637558
11,3,0.098069,0.086517,0.101042,49.970344
7,4,0.047396,0.108036,0.064936,49.366318
4,14,0.043717,0.085975,0.050136,23.464538


In [19]:
results_grid['hddc'].sort_values('silhouette_score_correlation', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
7,4,0.047396,0.108036,0.064936,49.366318
12,13,0.039065,0.106259,0.050703,23.317217
6,2,0.105046,0.100257,0.111406,67.637558
1,5,0.036925,0.090075,0.050502,39.756188


In [20]:
results_grid['hddc'].sort_values('silhouette_score_manhattan', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
6,2,0.105046,0.100257,0.111406,67.637558
11,3,0.098069,0.086517,0.101042,49.970344
7,4,0.047396,0.108036,0.064936,49.366318
10,7,0.043128,0.083040,0.053666,31.857787


In [21]:
results_grid['hddc'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score
6,2,0.105046,0.100257,0.111406,67.637558
11,3,0.098069,0.086517,0.101042,49.970344
7,4,0.047396,0.108036,0.064936,49.366318
1,5,0.036925,0.090075,0.050502,39.756188


In [23]:
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet1/Merged_GridHDDC.pkl', 'rb') as f:
    bic_df = pickle.load(f)
bic_df.head()

,model,K,threshold,LL,BIC,ICL,rank,complexity
12,1.0,2.0,0.01,-2.344247e+07,-5.304252e+07,-5.304252e+07,1,883109.0
30,1.0,3.0,0.10,-2.508192e+07,-5.372283e+07,-5.372283e+07,2,510424.0
13,1.0,2.0,0.10,-2.592310e+07,-5.376267e+07,-5.376267e+07,3,274856.0
24,1.0,4.0,0.10,-2.406654e+07,-5.442940e+07,-5.442940e+07,4,903009.0
31,1.0,3.0,0.20,-2.591699e+07,-5.443493e+07,-5.443493e+07,5,373024.0


# Combine

In [30]:
from bokeh.transform import factor_cmap, factor_mark
from bokeh.plotting import figure, show, output_file, save
from bokeh.models import ColumnDataSource, Dropdown, Select, Panel, Tabs, CustomJS, LinearAxis, Range1d, HoverTool
from bokeh.palettes import Category20
from bokeh.layouts import row, column
from bokeh.transform import cumsum

import panel as pn

In [62]:
def cluster_plot_bokeh(grid):
    line_dash = ['solid', 'dashed', 'dotted', 'dotdash']
    color = ['blue', 'red', 'hotpink', 'green']
    p = figure(width=1400, height=500, x_axis_label='Number clusters', y_axis_label='Harbrasz score')
    p.extra_y_ranges = {"foo1": Range1d(start=0, end=0.15)}
    p.add_layout(LinearAxis(y_range_name="foo1", axis_label='Silhouette score'), 'left')
    for i, alg in enumerate(grid):
        df = grid[alg].groupby(['n_clusters']).max().reset_index()
        df = df.sort_values('n_clusters')
        source = ColumnDataSource(df)
        l1 = p.line('n_clusters', 'calinski_harabasz_score', legend_label=f'calinski_harabasz_score {alg}',
                    source=source, line_dash=line_dash[0], color=color[i])
        p.add_tools(HoverTool(renderers=[l1], tooltips=[(f'Calinski Score {alg}',"@calinski_harabasz_score{0.00}"),
                                                        ('Clusters', '@n_clusters')],
                              mode='vline'))
        l2 = p.line('n_clusters', 'silhouette_score_euclidean', source=source,
                    y_range_name="foo1", legend_label=f'silhouette_score_euclidean {alg}', color=color[i],
                    line_dash=line_dash[1])
        p.add_tools(HoverTool(renderers=[l2], tooltips=[(f'Silhoette euclidian {alg}',"@silhouette_score_euclidean{0.00}")],
                              mode='vline'))
        l3 = p.line('n_clusters', 'silhouette_score_correlation', source=source,
                    y_range_name="foo1", legend_label=f'silhouette_score_correlation {alg}', color=color[i], 
                    line_dash=line_dash[2])
        p.add_tools(HoverTool(renderers=[l3], tooltips=[(f'Silhoette correlation {alg}',"@silhouette_score_correlation{0.00}")],
                              mode='vline'))
        l4 = p.line('n_clusters', 'silhouette_score_manhattan', source=source,
                    y_range_name="foo1", legend_label=f'silhouette_score_manhattan {alg}', color=color[i],
                    line_dash=line_dash[3])
        p.add_tools(HoverTool(renderers=[l4], tooltips=[(f'Silhoette manhattan {alg}',"@silhouette_score_manhattan{0.00}")],
                               mode='vline'))
    p.add_layout(p.legend[0], 'right')
    p.legend.click_policy="hide"
    return p

In [63]:
p = cluster_plot_bokeh(results_grid)
save(p, 'test.html')

'/home/g0017139/UMCG_Thesis/Working_Code/test.html'

In [75]:
end_df = pd.DataFrame()
for alg in results_grid:
    merge_df = results_grid[alg].copy()[['n_clusters','silhouette_score_euclidean','silhouette_score_correlation','silhouette_score_manhattan',
                                        'calinski_harabasz_score']].dropna()
    merge_df['Algorithm'] = alg
    end_df = end_df.append(merge_df)

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Algorithm
6,2,0.105046,0.100257,0.111406,67.637558,hddc
143,2,0.104322,0.099542,0.111301,66.925385,dbscan
3,2,0.104322,0.099542,0.111301,66.925385,kMeans
152,2,0.104322,0.099542,0.111301,66.925385,dbscan
150,2,0.104322,0.099542,0.111301,66.925385,dbscan


In [76]:
end_df.sort_values('silhouette_score_euclidean', ascending=False).head()

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Algorithm
6,2,0.105046,0.100257,0.111406,67.637558,hddc
143,2,0.104322,0.099542,0.111301,66.925385,dbscan
3,2,0.104322,0.099542,0.111301,66.925385,kMeans
152,2,0.104322,0.099542,0.111301,66.925385,dbscan
150,2,0.104322,0.099542,0.111301,66.925385,dbscan


In [77]:
end_df.sort_values('silhouette_score_correlation', ascending=False).head()

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Algorithm
5,11,0.037080,0.108922,0.047856,24.547031,Hierarch
7,4,0.047396,0.108036,0.064936,49.366318,hddc
9,4,0.049518,0.107492,0.066756,49.714021,kMeans
11,4,0.050610,0.106588,0.067730,49.595017,kMeans
10,4,0.050594,0.106517,0.067694,49.571423,kMeans


In [78]:
end_df.sort_values('silhouette_score_manhattan', ascending=False).head()

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Algorithm
6,2,0.105046,0.100257,0.111406,67.637558,hddc
152,2,0.104322,0.099542,0.111301,66.925385,dbscan
143,2,0.104322,0.099542,0.111301,66.925385,dbscan
158,2,0.104322,0.099542,0.111301,66.925385,dbscan
157,2,0.104322,0.099542,0.111301,66.925385,dbscan


In [79]:
end_df.sort_values('calinski_harabasz_score', ascending=False).head()

,n_clusters,silhouette_score_euclidean,silhouette_score_correlation,silhouette_score_manhattan,calinski_harabasz_score,Algorithm
6,2,0.105046,0.100257,0.111406,67.637558,hddc
0,2,0.102786,0.100446,0.109186,67.610557,kMeans
95,2,0.104322,0.099542,0.111301,66.925385,dbscan
120,2,0.104322,0.099542,0.111301,66.925385,dbscan
149,2,0.104322,0.099542,0.111301,66.925385,dbscan


# Set2

In [31]:
df, df_normalized, Type_df = load_sets(2)

## Hierarchical clustering

In [32]:
results_grid_2 = {}
clusters, results_grid_2['Hierarch'] =  merge_Results('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/', 'Hierarch', df_normalized)

GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample


In [33]:
results_grid_2['Hierarch'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
8,3,0.096387,1562.673504,3.259362
5,11,0.083546,975.211974,2.457754
13,2,0.083147,1653.816122,3.507638
4,12,0.080416,936.667096,2.494200


In [34]:
results_grid_2['Hierarch'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
13,2,0.083147,1653.816122,3.507638
8,3,0.096387,1562.673504,3.259362
3,5,0.048015,1305.012778,2.910270
10,4,0.029088,1280.769951,3.143995


In [35]:
get_score(df_normalized, clusters['3'].values)

GO Sample
GO Sample


{'silhouette_score': 0.08583627783284727,
 'calinski_harabasz_score': 1653.816121908958,
 'davies_bouldin_score': 3.5076380711316593}

In [36]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/MergedClustersHierarch.pkl', 'wb') as f:
    pickle.dump(clusters, f, protocol=pickle.HIGHEST_PROTOCOL)

## Kmeans

In [37]:
with open("/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/kmeans.pkl", 'rb') as f:
        results_grid_2['kMeans'] = pickle.load(f)

In [38]:
results_grid_2['kMeans'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,n_clusters,batch_size
5,0.132201,1875.684671,2.545246,122.278544,3,256
2,0.127093,2250.382308,2.835238,102.589599,2,512
3,0.125968,2172.172687,2.928323,133.569641,2,1024
0,0.123465,2208.600047,2.832577,46.080600,2,128


In [39]:
results_grid_2['kMeans'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,n_clusters,batch_size
1,0.120491,2282.028137,2.951187,99.034494,2,256
2,0.127093,2250.382308,2.835238,102.589599,2,512
0,0.123465,2208.600047,2.832577,46.080600,2,128
3,0.125968,2172.172687,2.928323,133.569641,2,1024


In [40]:
kmeans = MiniBatchKMeans(n_clusters=3,
                         random_state=0,
                         batch_size=256)
kmeans.fit(df_normalized)

MiniBatchKMeans(batch_size=256, n_clusters=3, random_state=0)

In [41]:
get_score(df_normalized, kmeans.labels_)

GO Sample
GO Sample


{'silhouette_score': 0.13061665238582404,
 'calinski_harabasz_score': 1875.6846708036805,
 'davies_bouldin_score': 2.5452457372669395}

## UMAP + DBSCAN

In [42]:
# with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/', 'rb') as f:
#         results_grid_2['dbscan'] = pickle.load(f)

In [43]:
#results_grid_2['dbscan'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

In [44]:
#results_grid_2['dbscan'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

In [45]:
# pipe = Pipeline([('DimReduction', 
#                   umap.UMAP(
#                       n_neighbors=30,
#                       min_dist=0.0,
#                       n_components=1,
#                       random_state=1)),
#                  ('Clustering', 
#                  hdbscan.HDBSCAN(min_cluster_size=50,
#                                 min_samples=75,
#                                 cluster_selection_epsilon=1.0,
#                                 cluster_selection_method='eom'))
#                 ])
# pipe.fit(df_normalized)

In [46]:
#get_score(df_normalized, pipe['Clustering'].labels_)

## HDDC

In [47]:
clusters, results_grid_2['hddc'], bic_df  =  merge_Results('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/', 'HDDC', df_normalized)

GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample


In [48]:
results_grid_2['hddc'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
1,5,0.120907,1322.542301,3.010284
8,3,0.113357,1693.289297,2.991432
4,2,0.104630,1871.341515,3.194748
5,4,0.096226,1243.688353,3.417905


In [49]:
results_grid_2['hddc'].sort_values('calinski_harabasz_score', axis=0, ascending=False).tail(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
2,8,0.069902,1155.281963,2.972268
7,7,0.056353,1143.419300,2.958209
0,9,0.065857,1042.710405,2.987186
6,10,0.077499,1023.927455,2.956921


In [50]:
bic_df.head()

,model,K,threshold,LL,BIC,ICL,rank,complexity
336,8.0,10.0,0.01,-3.361843e+08,-7.154549e+08,-7.154549e+08,1,4317230.0
0,8.0,8.0,0.01,-3.389497e+08,-7.170655e+08,-7.170655e+08,2,3924437.0
337,1.0,10.0,0.01,-3.385387e+08,-7.201656e+08,-7.201656e+08,3,4317411.0
294,12.0,9.0,0.01,-3.415283e+08,-7.230116e+08,-7.230116e+08,4,4003467.0
338,7.0,10.0,0.01,-3.400606e+08,-7.232094e+08,-7.232094e+08,5,4317420.0


In [51]:
get_score(df_normalized, clusters['10'].values)

GO Sample
GO Sample


{'silhouette_score': 0.07696662548998101,
 'calinski_harabasz_score': 1023.9274554453598,
 'davies_bouldin_score': 2.9569212067108146}

In [52]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/Merged_results.pkl', 'wb') as f:
    pickle.dump(results_grid_2, f, protocol=pickle.HIGHEST_PROTOCOL)

In [53]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/Merged_GridHDDC.pkl', 'wb') as f:
    pickle.dump(bic_df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set2/MergedClustersHDDC.pkl', 'wb') as f:
    pickle.dump(clusters, f, protocol=pickle.HIGHEST_PROTOCOL)

# Mixing set

In [8]:
df, df_normalized, Type_df = load_sets(3)

## Kmeans

In [9]:
results_grid_3 = {}

In [10]:
with open("/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/kmeans.pkl", 'rb') as f:
        results_grid_3['kMeans'] = pickle.load(f)

In [11]:
results_grid_3['kMeans'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,n_clusters,batch_size
2,0.149892,1333.994266,2.025419,0.074822,2,512
3,0.119751,1361.239351,2.132719,0.103506,2,1024
0,0.118935,1361.788266,2.140628,0.995648,2,128
4,0.117163,962.474313,2.177568,0.052269,3,128


In [12]:
results_grid_3['kMeans'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,n_clusters,batch_size
0,0.118935,1361.788266,2.140628,0.995648,2,128
3,0.119751,1361.239351,2.132719,0.103506,2,1024
1,0.112108,1355.976603,2.160696,0.094072,2,256
2,0.149892,1333.994266,2.025419,0.074822,2,512


In [13]:
kmeans = MiniBatchKMeans(n_clusters=2,
                         random_state=0,
                         batch_size=512)
kmeans.fit(df_normalized)

MiniBatchKMeans(batch_size=512, n_clusters=2, random_state=0)

In [14]:
get_score(df_normalized, kmeans.labels_)

{'silhouette_score': 0.14989164962724896,
 'calinski_harabasz_score': 1333.994266220463,
 'davies_bouldin_score': 2.025418652841891}

## Hierachical

In [15]:
clusters, results_grid_3['hierarch'] =  merge_Results('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/', 'Hierarch', df_normalized)

In [16]:
results_grid_3['hierarch'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
10,4,0.134237,765.642918,2.004267
8,3,0.131812,737.023991,2.470519
13,2,0.125887,815.872864,2.299241
9,6,0.064162,627.242106,2.471408


In [17]:
results_grid_3['hierarch'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
13,2,0.125887,815.872864,2.299241
10,4,0.134237,765.642918,2.004267
8,3,0.131812,737.023991,2.470519
3,5,0.059688,689.817782,2.443426


In [18]:
get_score(df_normalized, clusters['2'])

{'silhouette_score': 0.12588674056625399,
 'calinski_harabasz_score': 815.8728638997788,
 'davies_bouldin_score': 2.299240732627995}

In [19]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/MergedClustersHierarch.pkl', 'wb') as f:
    pickle.dump(clusters, f, protocol=pickle.HIGHEST_PROTOCOL)

## UMAP + DBSCAN

In [20]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/dbscan.pkl', 'rb') as f:
        results_grid_3['dbscan'] = pickle.load(f)

In [21]:
results_grid_3['dbscan'].sort_values('silhouette_score', axis=0, ascending=False).head()

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
740,0.340171,629.62311,1.530836,38.771169,50,0.5,10,100,25,0.5,eom,2
736,0.340171,629.62311,1.530836,38.924523,50,0.5,10,50,100,1.0,eom,2
730,0.340171,629.62311,1.530836,42.138569,50,0.5,10,50,50,1.0,eom,2
268,0.340171,629.62311,1.530836,27.277199,25,0.5,10,100,100,1.0,eom,2
732,0.340171,629.62311,1.530836,40.799728,50,0.5,10,50,100,0.1,eom,2


In [22]:
results_grid_3['dbscan'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
808,0.340171,629.62311,1.530836,39.087287,50,0.5,2,100,100,1.0,eom,2
750,0.340171,629.62311,1.530836,40.780310,50,0.5,10,100,100,0.1,eom,2
754,0.340171,629.62311,1.530836,43.609672,50,0.5,10,100,100,1.0,eom,2
1254,0.340171,629.62311,1.530836,46.790243,100,0.5,2,25,100,0.1,eom,2


In [23]:
pipe = Pipeline([('DimReduction', 
                  umap.UMAP(
                      n_neighbors=50,
                      min_dist=0.5,
                      n_components=50,
                      random_state=42)),
                 ('Clustering', 
                 hdbscan.HDBSCAN(min_cluster_size=25,
                                min_samples=100,
                                cluster_selection_epsilon=1.0,
                                cluster_selection_method='eom'))
                ])
pipe.fit(df_normalized)

Pipeline(steps=[('DimReduction',
                 UMAP(dens_frac=0.0, dens_lambda=0.0, min_dist=0.5,
                      n_components=50, n_neighbors=50, random_state=42)),
                ('Clustering',
                 HDBSCAN(cluster_selection_epsilon=1.0, min_cluster_size=25,
                         min_samples=100))])

In [24]:
get_score(df_normalized, pipe['Clustering'].labels_)

{'silhouette_score': 0.34017073834245265,
 'calinski_harabasz_score': 629.6231101637583,
 'davies_bouldin_score': 1.530836481901089}

## HDDC

In [27]:
clusters, results_grid_3['hddc'], bic_df =  merge_Results('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/', 'HDDC', df_normalized)

In [28]:
results_grid_3['hddc'].sort_values('silhouette_score', axis=0, ascending=False).head()

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
3,8,0.064705,498.799162,2.371565
5,6,0.055544,475.664813,2.912394
8,10,0.053181,429.416490,2.687975
9,12,0.038021,386.022327,2.867803
7,4,0.033622,348.701800,2.959841


In [29]:
results_grid_3['hddc'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
3,8,0.064705,498.799162,2.371565
5,6,0.055544,475.664813,2.912394
1,5,0.022890,468.324839,2.773197
10,7,0.030595,447.758197,2.896149


In [30]:
bic_df.head(4)

,model,K,threshold,LL,BIC,ICL,rank,complexity,comment
1092,12.0,10.0,0.01,7.586122e+06,1.428560e+07,1.428542e+07,1,99972.0,0
504,8.0,6.0,0.01,7.369402e+06,1.426302e+07,1.426295e+07,2,53646.0,0
378,8.0,7.0,0.01,7.417999e+06,1.426141e+07,1.426132e+07,3,64786.0,0
505,12.0,7.0,0.01,7.403864e+06,1.425936e+07,1.425930e+07,4,61830.0,0


In [31]:
get_score(df_normalized, clusters['2'])

{'silhouette_score': 0.024128409722691153,
 'calinski_harabasz_score': 396.54654228173393,
 'davies_bouldin_score': 3.6515097695441505}

In [32]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/Merged_results.pkl', 'wb') as f:
    pickle.dump(results_grid_3, f, protocol=pickle.HIGHEST_PROTOCOL)

In [33]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/Merged_GridHDDC.pkl', 'wb') as f:
    pickle.dump(bic_df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set3/MergedClustersHDDC.pkl', 'wb') as f:
    pickle.dump(clusters, f, protocol=pickle.HIGHEST_PROTOCOL)

# Set3

In [35]:
df, df_normalized, Type_df = load_sets(4)

## Hierarchical clustering

In [37]:
results_grid_4 = {}
clusters, results_grid_4['Hierarch'] =  merge_Results('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/', 'Hierarch', df_normalized)

GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample


In [38]:
results_grid_4['Hierarch'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
7,14,0.070960,202.337064,3.622430
12,15,0.066621,193.715501,3.592057
0,13,0.065438,208.544113,3.489262
8,3,0.062752,363.632244,4.465577


In [39]:
results_grid_4['Hierarch'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
8,3,0.062752,363.632244,4.465577
13,2,0.053204,328.626991,5.395752
10,4,0.047767,319.223193,4.226990
3,5,0.039610,282.414376,4.178298


In [40]:
get_score(df_normalized, clusters['2'])

GO Sample
GO Sample


{'silhouette_score': 0.05459389405738467,
 'calinski_harabasz_score': 328.6269912164971,
 'davies_bouldin_score': 5.395752485891356}

In [41]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/MergedClustersHierarch.pkl', 'wb') as f:
    pickle.dump(clusters, f, protocol=pickle.HIGHEST_PROTOCOL)

## Kmeans

In [43]:
with open("/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/kmeans.pkl", 'rb') as f:
        results_grid_4['kMeans'] = pickle.load(f)

In [44]:
results_grid_4['kMeans'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,n_clusters,batch_size
1,0.301160,285.554676,1.604930,3.765494,2,256
3,0.146398,427.135577,2.535278,7.665748,2,1024
2,0.108926,355.253090,4.509859,9.856236,2,512
10,0.094110,314.124875,3.388982,14.794719,4,512


In [45]:
results_grid_4['kMeans'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,n_clusters,batch_size
3,0.146398,427.135577,2.535278,7.665748,2,1024
0,0.067395,406.932042,4.833783,6.548981,2,128
7,0.066097,389.851276,4.054005,8.252044,3,1024
9,0.077487,363.298433,3.879248,3.787388,4,256


In [46]:
kmeans = MiniBatchKMeans(n_clusters=2,
                         random_state=0,
                         batch_size=1024)
kmeans.fit(df_normalized)

MiniBatchKMeans(batch_size=1024, n_clusters=2, random_state=0)

In [47]:
get_score(df_normalized, kmeans.labels_)

GO Sample
GO Sample


{'silhouette_score': 0.15201761458507923,
 'calinski_harabasz_score': 427.13557667184295,
 'davies_bouldin_score': 2.5352782783036907}

## DBSCAN + UMAP

In [48]:
with open("/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/dbscan.pkl", 'rb') as f:
        results_grid_4['dbscan'] = pickle.load(f)

In [49]:
results_grid_4['dbscan'].sort_values('silhouette_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
856,0.320121,285.554676,1.60493,305.186532,50,1.0,50,100,50,1.0,eom,2
820,0.319187,285.554676,1.60493,313.165519,50,1.0,50,25,50,1.0,eom,2
886,0.317110,285.554676,1.60493,162.304850,50,1.0,10,50,25,1.0,eom,2
356,0.314472,285.554676,1.60493,188.696043,25,1.0,50,50,100,0.5,eom,2


In [50]:
results_grid_4['dbscan'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,silhouette_score,calinski_harabasz_score,davies_bouldin_score,Fit_Time,DimReduction__n_neighbors,DimReduction__min_dist,DimReduction__n_components,Clustering__min_cluster_size,Clustering__min_samples,Clustering__cluster_selection_epsilon,Clustering__cluster_selection_method,n_clusters
1420,0.143368,392.656581,2.255541,640.879837,100,1.0,2,25,100,1.0,eom,2
1430,0.154656,392.436666,2.256462,645.837430,100,1.0,2,50,50,0.5,eom,2
1218,0.154278,349.967024,2.021050,755.840429,100,0.5,10,50,100,0.1,eom,3
1146,0.152783,349.967024,2.021050,351.981237,100,0.5,50,25,100,0.1,eom,3


In [ ]:
# pipe = Pipeline([('DimReduction', 
#                   umap.UMAP(
#                       n_neighbors=50,
#                       min_dist=1.0,
#                       n_components=50,
#                       random_state=1)),
#                  ('Clustering', 
#                  hdbscan.HDBSCAN(min_cluster_size=50,
#                                 min_samples=25,
#                                 cluster_selection_epsilon=1.0,
#                                 cluster_selection_method='eom'))
#                 ])
# pipe.fit(df_normalized)

In [ ]:
#get_score(df_normalized, pipe['Clustering'].labels_)

## HDDC

In [51]:
clusters, results_grid_4['hddc'], bic_df =  merge_Results('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/', 'HDDC', df_normalized)

GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample
GO Sample


In [52]:
results_grid_4['hddc'].sort_values('silhouette_score', axis=0, ascending=False).head()

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
11,15,0.050439,198.537141,3.816309
6,10,0.041570,211.003360,4.081448
4,2,0.029447,138.204736,8.302046
7,12,0.027156,196.569313,4.820216
10,13,0.024551,181.124951,4.929371


In [53]:
results_grid_4['hddc'].sort_values('calinski_harabasz_score', axis=0, ascending=False).head(4)

,n_clusters,silhouette_score,calinski_harabasz_score,davies_bouldin_score
5,4,0.009053,247.434559,6.383849
8,7,0.009325,243.171677,4.803565
3,6,0.020172,237.972790,4.986615
6,10,0.041570,211.003360,4.081448


In [59]:
bic_df.head(4)

,model,K,threshold,LL,BIC,ICL,rank,complexity,comment,1


In [55]:
get_score(df_normalized, clusters['2'])

GO Sample
GO Sample


{'silhouette_score': 0.026905198615828282,
 'calinski_harabasz_score': 138.20473635667233,
 'davies_bouldin_score': 8.302045893487888}

In [56]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/Merged_results.pkl', 'wb') as f:
    pickle.dump(results_grid_4, f, protocol=pickle.HIGHEST_PROTOCOL)

In [57]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/Merged_GridHDDC.pkl', 'wb') as f:
    pickle.dump(bic_df, f, protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
with open('/home/g0017139/UMCG_Thesis/Working_Code/Results/Set4/MergedClustersHDDC.pkl', 'wb') as f:
    pickle.dump(clusters, f, protocol=pickle.HIGHEST_PROTOCOL)